# Rich Logging and Printing

In [1]:
# default_exp logger

In [2]:
# export
from rich.console import Console
from rich.theme import Theme


def get_console(width=None):
    return Console(
        width=width,
        theme=Theme(
            {
                "repr.number": "bold cyan",
                "repr.string": "bold green",
                "logging.level.info": "dim yellow",
                "logging.level.warning": "dim red",
                "logging.level.exception": "bold red",
            }
        ),
    )


console = get_console()
print = console.print

In [3]:
print("The Number is 128")
print(
    {
        "a": 1,
        "b": [
            {1, 2, 3},
            "lskjdf",
        ],
        "c": 1,
        "d": [
            {1, 2, 3},
            "lskjdf",
        ],
        "ae": 1,
        "bf": [
            {1, 2, 3},
            "lskjdf",
        ],
        "ag": 1,
        "ba": [
            {1, 2, 3},
            "lskjdf",
        ],
    }
)

The Number is 128

{
    'a': 1,
    'b': [{1, 2, 3}, 'lskjdf'],
    'c': 1,
    'd': [{1, 2, 3}, 'lskjdf'],
    'ae': 1,
    'bf': [{1, 2, 3}, 'lskjdf'],
    'ag': 1,
    'ba': [{1, 2, 3}, 'lskjdf']
}

In [4]:
print(r"\x86")

\x86

In [5]:
# export
from loguru import logger
from datetime import datetime
from fastcore.basics import patch_to
from rich.logging import RichHandler
from pathlib import Path


@patch_to(RichHandler)
def render(
    self,
    *,
    record,
    traceback,
    message_renderable: "ConsoleRenderable",
) -> "ConsoleRenderable":
    """patched the renderer to print function name as well"""
    path = Path(record.pathname).name
    level = self.get_level_text(record)
    time_format = None if self.formatter is None else self.formatter.datefmt
    log_time = datetime.fromtimestamp(record.created)

    log_renderable = self._log_render(
        self.console,
        [message_renderable] if not traceback else [message_renderable, traceback],
        log_time=log_time,
        time_format=time_format,
        level=level,
        path=path,
        line_no=f"{record.funcName}:{record.lineno}",
        link_path=record.pathname if self.enable_link_path else None,
    )
    return log_renderable


logger.configure(handlers=[{
    "sink":RichHandler(
        rich_tracebacks=True,
        console=console,
        tracebacks_show_locals=False
    ),
    "format":'<level>{message}</level>',
    "backtrace": True
}])

logger = logger

Debug = lambda x, depth=0: logger.opt(depth=depth + 1).log("DEBUG", x)
Info = lambda x, depth=0: logger.opt(depth=depth + 1).log("INFO", x)
Warn = lambda x, depth=0: logger.opt(depth=depth + 1).log("WARNING", x)
Excep = lambda x, depth=0: logger.opt(depth=depth + 1).log("ERROR", x)

In [6]:
a = {1: 1221, 2: 2342}
del a[1]
a

{2: 2342}

In [7]:
Debug("TESTING {1,2,3}")
Info("TESTING {1,2,3}")
Warn("TESTING {1,2,3}")
Excep("TESTING {1,2,3}")

[08/28/22 20:15:35] DEBUG    TESTING {1,2,3}                         ]8;id=240023;file:///var/folders/cp/1fbgq2n922j8ztdsq6551vldkr5sdy/T/ipykernel_28554/3592958886.py\3592958886.py]8;;\:]8;id=48006;file:///var/folders/cp/1fbgq2n922j8ztdsq6551vldkr5sdy/T/ipykernel_28554/3592958886.py#<module>:1\<module>:1]8;;\

                    INFO     TESTING {1,2,3}                         ]8;id=40153;file:///var/folders/cp/1fbgq2n922j8ztdsq6551vldkr5sdy/T/ipykernel_28554/3592958886.py\3592958886.py]8;;\:]8;id=777439;file:///var/folders/cp/1fbgq2n922j8ztdsq6551vldkr5sdy/T/ipykernel_28554/3592958886.py#<module>:2\<module>:2]8;;\

                    WARNING  TESTING {1,2,3}                         ]8;id=974310;file:///var/folders/cp/1fbgq2n922j8ztdsq6551vldkr5sdy/T/ipykernel_28554/3592958886.py\3592958886.py]8;;\:]8;id=324975;file:///var/folders/cp/1fbgq2n922j8ztdsq6551vldkr5sdy/T/ipykernel_28554/3592958886.py#<module>:3\<module>:3]8;;\

                    ERROR    TESTING {1,2,3}                         ]8;id=427623;file:///var/folders/cp/1fbgq2n922j8ztdsq6551vldkr5sdy/T/ipykernel_28554/3592958886.py\3592958886.py]8;;\:]8;id=794297;file:///var/folders/cp/1fbgq2n922j8ztdsq6551vldkr5sdy/T/ipykernel_28554/3592958886.py#<module>:4\<module>:4]8;;\

In [8]:
# export
from torch_snippets.ipython import is_in_notebook

def reset_logger_width(logger, width):
    for handler_id in logger._core.handlers:
        try:
            handler = logger._core.handlers[handler_id]
            handler._sink._handler.console = get_console(width=width)
            logger.info(f"reset logger's console width to {width}!")
        except:
            ...

# Excep("TESTING {1,2,3}")
# if is_in_notebook():
#     reset_logger_width(logger, 115)
# Excep("TESTING {1,2,3}")

In [9]:
# exprimental
from functools import wraps
import time


def enter_exit(func):
    """
    Logs when a function
    """
    logger_ = logger.opt(depth=1)

    @wraps(func)
    def function_timer(*args, **kwargs):
        tic = time.time()
        logger_.log("DEBUG", f"Entered function `{func.__name__}`")
        o = func(*args, **kwargs)
        toc = time.time()
        logger_.log(
            "DEBUG", f"Exiting function `{func.__name__}` after {toc-tic:.3f} seconds"
        )
        return o

    return function_timer

In [10]:
@enter_exit
def add(x, y):
    print("sleeping...")
    time.sleep(2)
    return x + y


add(1, 23)

[08/28/22 20:15:38] DEBUG    Entered function `add`                  ]8;id=652976;file:///var/folders/cp/1fbgq2n922j8ztdsq6551vldkr5sdy/T/ipykernel_28554/4246827219.py\4246827219.py]8;;\:]8;id=92059;file:///var/folders/cp/1fbgq2n922j8ztdsq6551vldkr5sdy/T/ipykernel_28554/4246827219.py#<module>:8\<module>:8]8;;\

sleeping...

[08/28/22 20:15:40] DEBUG    Exiting function `add` after 2.004      ]8;id=761781;file:///var/folders/cp/1fbgq2n922j8ztdsq6551vldkr5sdy/T/ipykernel_28554/4246827219.py\4246827219.py]8;;\:]8;id=842060;file:///var/folders/cp/1fbgq2n922j8ztdsq6551vldkr5sdy/T/ipykernel_28554/4246827219.py#<module>:8\<module>:8]8;;\
                             seconds

24

In [11]:
def do():
    try:
        1 / 0
    except Exception as e:
        # console.print_exception(max_frames=20)
        logger.exception(e)
        
def do2():
    do()
    
do2()

                    ERROR    division by zero                              ]8;id=571179;file:///var/folders/cp/1fbgq2n922j8ztdsq6551vldkr5sdy/T/ipykernel_28554/3806461673.py\3806461673.py]8;;\:]8;id=740507;file:///var/folders/cp/1fbgq2n922j8ztdsq6551vldkr5sdy/T/ipykernel_28554/3806461673.py#do:6\do:6]8;;\
                             Traceback (most recent call last):                              
                                                                                             
                               File "/var/folders/cp/1fbgq2n922j8ztdsq6551                   
                             vldkr5sdy/T/ipykernel_28554/3806461673.py",                     
                             line 3, in do                                                   
                                 1 / 0                                                       
                                                                                             
                             ZeroDivisionError: division by zero